In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv("/kaggle/input/cvhackathon/train.csv")
test = pd.read_csv("/kaggle/input/cvhackathon/test_vc2kHdQ.csv")
sample = pd.read_csv("/kaggle/input/cvhackathon/sample_submission_yxjOnvz.csv")
img_dir = "/kaggle/input/images/images/"

In [ ]:
y = train['emergency_or_not']

In [ ]:
#convert('L') to convert to gray scale
from PIL import Image
train_images=np.array(train.iloc[:,0])
test_images=np.array(test.iloc[:,0])
trainimagearr=[]
for i in train_images:
    img=Image.open(img_dir+i).resize((224,224))
    trainimagearr.append(np.array(img))
testimagearr=[]
for i in test_images:
    img=Image.open(img_dir+i).resize((224,224))
    testimagearr.append(np.array(img))

In [ ]:
train_img=np.array(trainimagearr)
print(train_img.shape)
test_img=np.array(testimagearr)
print(test_img.shape)

In [ ]:
plt.imshow(train_img[10])

In [ ]:
import os
dir_train_emer = 'train/emergency'
dir_train_non_emer = 'train/not_emergency'

dir_valid_emer = 'valid/emergency'
dir_valid_non_emer = 'valid/not_emergency'

parent_dir = 'cv'

path1 = os.path.join(parent_dir, dir_train_emer)
os.makedirs(path1, exist_ok=True)

path1 = os.path.join(parent_dir, dir_train_non_emer)
os.makedirs(path1, exist_ok=True)

path1 = os.path.join(parent_dir, dir_valid_emer)
os.makedirs(path1, exist_ok=True)

path1 = os.path.join(parent_dir, dir_valid_non_emer)
os.makedirs(path1, exist_ok=True)

In [ ]:
for x in train.query('emergency_or_not==1')['image_names']:
    path2 = os.path.join(img_dir, x)
    img = Image.open(path2)
    path_emer=os.path.join(parent_dir, dir_train_emer)
    path_emer=os.path.join(path_emer,x)
    img.save(path_emer)
    
for x in train.query('emergency_or_not==0')['image_names']:
    path2 = os.path.join(img_dir, x)
    img = Image.open(path2)
    path_emer=os.path.join(parent_dir, dir_train_non_emer)
    path_emer=os.path.join(path_emer,x)
    img.save(path_emer)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
import os
import keras
import numpy as np
import pandas as pd
from zipfile import ZipFile
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

# VGG16

In [ ]:
from keras import applications
from keras.utils.np_utils import to_categorical
import math

img_width, img_height = 224,224
epochs = 50
batch_size = 64

vgg16 = applications.VGG16(include_top = False, weights='imagenet')
datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_set = datagen.flow_from_directory('cv/train', target_size=(img_width, img_height), batch_size=batch_size, class_mode=None, shuffle=False)

nb_train_samples = len(train_set.filenames) 
num_classes = len(train_set.class_indices)  
predict_size_train = int(math.ceil(nb_train_samples / batch_size))

train_features = vgg16.predict_generator(train_set, predict_size_train)
np.save('train_features.npy', train_features)

train_set_labels = datagen.flow_from_directory('cv/train', target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary', shuffle=False)

nb_train_samples = len(train_set_labels.filenames) 
num_classes = len(train_set_labels.class_indices)  

In [ ]:
train_data = np.load('train_features.npy')  
train_labels = train_set_labels.classes  
train_labels = to_categorical(train_labels, num_classes=num_classes)

In [ ]:
vggclassifier = Sequential() 
vggclassifier.add(Flatten(input_shape=train_data.shape[1:])) 

vggclassifier.add(Dense(128, activation = 'relu', kernel_initializer='he_uniform')) # he_uniform
vggclassifier.add(Dropout(0.3))

vggclassifier.add(Dense(64, activation = 'relu', kernel_initializer='he_uniform')) # he_uniform
vggclassifier.add(Dropout(0.3))

vggclassifier.add(Dense(num_classes, activation = 'softmax', kernel_initializer='he_uniform')) # he_uniform
vggclassifier.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
vggclassifier.fit(train_data, train_labels, epochs=150, batch_size=batch_size)

In [ ]:
from keras.preprocessing import image

main_result = []

for x in test['image_names']:
    path2 = os.path.join(img_dir, x)
    test_image = image.load_img(path2, target_size=(img_height, img_width))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    bt_prediction = vgg16.predict(test_image)
    result = vggclassifier.predict(bt_prediction)
    print(result)
    main_result.append(result[0][0])

In [ ]:
sample['emergency_or_not'] = main_result
sample['emergency_or_not'] = np.where(sample['emergency_or_not']<0.6, 0, 1)
sample

In [ ]:
sample.to_csv('subvgg14.csv', index=False)

# ResNet50

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 224
WIDTH = 224

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

HEIGHT = 224
WIDTH = 224
BATCH_SIZE = 64

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      shear_range=0.2, 
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory('cv/train', 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)

In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

FC_LAYERS = [128, 64]
dropout = 0.3

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=2)

In [ ]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

NUM_EPOCHS = 150
BATCH_SIZE = 32

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

# filepath="./checkpoints/" + "ResNet50" + "_model_weights.h5"
# checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
# callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, 
                                       shuffle=True)

In [ ]:
from keras.preprocessing import image

main_result = []

for x in test['image_names']:
    path2 = os.path.join(img_dir, x)
    test_image = image.load_img(path2, target_size=(HEIGHT, WIDTH))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = finetune_model.predict(test_image)
    print(result)
    main_result.append(result[0][0])

In [ ]:
sample['emergency_or_not'] = main_result
sample['emergency_or_not'] = np.where(sample['emergency_or_not']<0.80, 0, 1)
sample

In [ ]:
sample.to_csv('submres3.csv', index=False)

# Inception

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# add a global spatial average pooling layer
x = base_model.output
x = Flatten()(x)
# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, epochs=100, shuffle=True)

In [ ]:
from keras.preprocessing import image

main_result = []

for x in test['image_names']:
    path2 = os.path.join(img_dir, x)
    test_image = image.load_img(path2, target_size=(HEIGHT, WIDTH))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image)
    print(result)
    main_result.append(result[0][0])

In [ ]:
sample['emergency_or_not'] = main_result
sample['emergency_or_not'] = np.where(sample['emergency_or_not']<1, 0, 1)
sample

In [ ]:
sample.to_csv("inception.csv", index=False)

# Ensembling

In [ ]:
models = [vggclassifier, finetune_model, model]

In [ ]:
from keras.preprocessing import image

main_result = []

for x in test['image_names']:
    path2 = os.path.join(img_dir, x)
    test_image = image.load_img(path2, target_size=(HEIGHT, WIDTH))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    bt_prediction = vgg16.predict(test_image)
    result = vggclassifier.predict(bt_prediction)[0][0]
    pred = [result]
    pred.append(finetune_model.predict(test_image)[0][0])
    pred.append(model.predict(test_image)[0][0])
    pred = np.array(pred)
    if np.sum(pred, axis=0)>0.8:
        result = 1
    else:
        result = 0
    main_result.append(result)

In [ ]:
sample['emergency_or_not'] = main_result
# sample['emergency_or_not'] = np.where(sample['emergency_or_not']<0.90, 0, 1)
sample

In [ ]:
sample.to_csv('ensemble11.csv', index=False)